In [1]:
import AlGDock.BindingPMF_plots
from AlGDock.BindingPMF_plots import *
import os, shutil, glob

phases = ['NAMD_Gas', 'NAMD_OBC']

self = AlGDock.BindingPMF_plots.BPMF_plots(\
  dir_dock='dock', dir_cool='cool',\
  ligand_database='prmtopcrd/ligand.db', \
  forcefield='prmtopcrd/gaff2.dat', \
  ligand_prmtop='prmtopcrd/ligand.prmtop', \
  ligand_inpcrd='prmtopcrd/ligand.trans.inpcrd', \
  ligand_mol2='prmtopcrd/ligand.mol2', \
  ligand_rb='prmtopcrd/ligand.rb', \
  receptor_prmtop='prmtopcrd/receptor.prmtop', \
  receptor_inpcrd='prmtopcrd/receptor.trans.inpcrd', \
  receptor_fixed_atoms='prmtopcrd/receptor.pdb', \
  complex_prmtop='prmtopcrd/complex.prmtop', \
  complex_inpcrd='prmtopcrd/complex.trans.inpcrd', \
  complex_fixed_atoms='prmtopcrd/complex.pdb', \
  score = 'prmtopcrd/xtal_plus_dock6_scored.mol2', \
  temperature_scaling = 'Quadratic', \
  pose = -1, \
  rmsd=True, \
  dir_grid='grids', \
  protocol='Adaptive', cool_therm_speed=25.0, dock_therm_speed=0.25, \
  T_HIGH=450.0, T_SIMMIN=300.0, T_TARGET=300.0, \
  sampler='HMC', \
  MCMC_moves=1, \
  sampling_importance_resampling = True, \
  solvation = 'Full', \
  seeds_per_state=10, steps_per_seed=200, darts_per_seed=0, \
  sweeps_per_cycle=50, snaps_per_cycle=25, attempts_per_sweep=100, \
  steps_per_sweep=50, darts_per_sweep=0, \
  cool_repX_cycles=3, dock_repX_cycles=4, \
  site='Sphere', site_center=[1.7416, 1.7416, 1.7416], \
  site_max_R=1.0, \
  site_density=10., \
  phases=phases, \
  cores=-1, \
  random_seed=-1, \
  max_time=240, \
  keep_intermediate=True)

###########
# AlGDock #
###########
Molecular docking with adaptively scaled alchemical interaction grids

in /Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/AlGDock/BindingPMF.py
last modified Tue Aug  8 13:56:54 2017
    
using 4/4 available cores
using random number seed of -1

*** Directories ***
  start: /Users/dminh/Installers/AlGDock-0.0.1/Example
  cool: /Users/dminh/Installers/AlGDock-0.0.1/Example/cool
  dock: /Users/dminh/Installers/AlGDock-0.0.1/Example/dock

*** Files ***
previously stored in cool directory:
  prmtop:
    L: prmtopcrd/ligand.prmtop
  ligand_database: prmtopcrd/ligand.db
  forcefield: ../Data/gaff2.dat
  inpcrd:
    L: prmtopcrd/ligand.trans.inpcrd

previously stored in dock directory:
  namd: ../../../Applications/NAMD_2.10/namd2
  grids:
    LJr: grids/LJr.nc
    LJa: grids/LJa.nc
    ELE: grids/pbsa.nc
    desolv: grids/desolv.nc
  prmtop:
    L: prmtopcrd/ligand.prmtop
    R: prmtopcrd/receptor.prmtop
    RL: prmtopcrd/comp

In [2]:
self._run('all')


Elapsed time for execution of all: 0.13 s


In [4]:
process = 'dock'

In [5]:
    self._set_lock(process)

    cycle = getattr(self,'_%s_cycle'%process)
    confs = self.confs[process]['replicas']
    lambdas = getattr(self,process+'_protocol')

    terms = ['MM']
    if process=='cool':
      terms += ['OBC']
    elif process=='dock':
      if self.params['dock']['pose'] > -1:
        # Pose BPMF
        terms += ['k_angular_ext','k_spatial_ext','k_angular_int']
      else:
        terms += ['site']
      terms += self._scalables

    # A list of pairs of replica indicies
    K = len(lambdas)
    pairs_to_swap = []
    for interval in range(1,min(5,K)):
      lower_inds = []
      for lowest_index in range(interval):
        lower_inds += range(lowest_index,K-interval,interval)
      upper_inds = np.array(lower_inds) + interval
      pairs_to_swap += zip(lower_inds,upper_inds)

    from repX import attempt_swaps

    # Setting the force field will load grids
    # before multiple processes are spawned
    for k in range(K):
      self._set_universe_evaluator(lambdas[k])
    
    # If it has not been set up, set up Smart Darting
    if self.params[process]['darts_per_sweep']>0:
      if self.sampler[process+'_SmartDarting'].confs==[]:
        self.tee(self.sampler[process+'_SmartDarting'].set_confs(\
          self.confs[process]['SmartDarting']))
        self.confs[process]['SmartDarting'] = \
          self.sampler[process+'_SmartDarting'].confs
  
    # storage[key][sweep_index][state_index] will contain data
    # from the replica exchange sweeps
    storage = {}
    for var in ['confs','state_inds','energies']:
      storage[var] = []
    
    self.start_times['repX cycle'] = time.time()

    if self._cores>1:
      # Multiprocessing setup
      m = multiprocessing.Manager()
      task_queue = m.Queue()
      done_queue = m.Queue()

    # GMC
    do_gMC = self.params[process]['GMC_attempts'] > 0
    if do_gMC:
      self.tee('  Using GMC for %s' %process)
      nr_gMC_attempts = K * self.params[process]['GMC_attempts']
      torsion_threshold = self.params[process]['GMC_tors_threshold']
      gMC_attempt_count = 0
      gMC_acc_count     = 0
      time_gMC = 0.0
      BAT_converter, state_indices_to_swap = gMC_initial_setup()

    # MC move statistics
    acc = {}
    att = {}
    for move_type in ['ExternalMC','SmartDarting','Sampler']:
      acc[move_type] = np.zeros(K, dtype=int)
      att[move_type] = np.zeros(K, dtype=int)
      self.timings[move_type] = 0.
    self.timings['repX'] = 0.
    
    mean_energies = []

    # Do replica exchange
    state_inds = range(K)
    inv_state_inds = range(K)
    nsweeps = self.params[process]['sweeps_per_cycle']
    for sweep in range(nsweeps):
      E = {}
      for term in terms:
        E[term] = np.zeros(K, dtype=float)
      # Sample within each state
      if self._cores>1:
        for k in range(K):
          task_queue.put((confs[k], process, lambdas[state_inds[k]], False, k))
        for p in range(self._cores):
          task_queue.put('STOP')
        processes = [multiprocessing.Process(target=self._sim_one_state_worker, \
            args=(task_queue, done_queue)) for p in range(self._cores)]
        for p in processes:
          p.start()
        for p in processes:
          p.join()
        unordered_results = [done_queue.get() for k in range(K)]
        results = sorted(unordered_results, key=lambda d: d['reference'])
        for p in processes:
          p.terminate()
      else:
        # Single process code
        results = [self._sim_one_state(confs[k], process, \
            lambdas[state_inds[k]], False, k) for k in range(K)]

      # GMC
      if do_gMC:
        time_start_gMC = time.time()
        att_count, acc_count = do_gMC( nr_gMC_attempts, BAT_converter, state_indices_to_swap, torsion_threshold )
        gMC_attempt_count += att_count
        gMC_acc_count     += acc_count
        time_gMC =+ ( time.time() - time_start_gMC )

      # Store energies
      for k in range(K):
        confs[k] = results[k]['confs']
      mean_energies.append(np.mean([results[k]['Etot'] for k in range(K)]))
      E = self._energyTerms(confs, E, process=process)
      if (process=='dock') and (self.params['dock']['rmsd'] is not False):
        E['rmsd'] = np.array([np.sqrt(((confs[k][self.molecule.heavy_atoms,:] - \
          self.confs['rmsd'])**2).sum()/self.molecule.nhatoms) for k in range(K)])

      # Store MC move statistics
      for k in range(K):
        for move_type in ['ExternalMC','SmartDarting','Sampler']:
          key = 'acc_'+move_type
          if key in results[k].keys():
            acc[move_type][state_inds[k]] += results[k][key]
            att[move_type][state_inds[k]] += results[k]['att_'+move_type]
            self.timings[move_type] += results[k]['time_'+move_type]

      # Calculate u_ij (i is the replica, and j is the configuration),
      #    a list of arrays
      (u_ij,N_k) = self._u_kln(E, [lambdas[state_inds[c]] for c in range(K)])
      # Do the replica exchange
      repX_start_time = time.time()
      (state_inds, inv_state_inds) = \
        attempt_swaps(state_inds, inv_state_inds, u_ij, pairs_to_swap, \
          self.params[process]['attempts_per_sweep'])
      self.timings['repX'] += (time.time()-repX_start_time)

      # Store data in local variables
      storage['confs'].append(list(confs))
      storage['state_inds'].append(list(state_inds))
      storage['energies'].append(copy.deepcopy(E))

    # GMC
    if do_gMC:
      self.tee('  {0}/{1} crossover attempts ({2:.3g}) accepted in {3}'.format(\
        gMC_acc_count, gMC_attempt_count, \
        float(gMC_acc_count)/float(gMC_attempt_count) \
          if gMC_attempt_count > 0 else 0, \
        HMStime(time_gMC)))

    # Report
    self.tee("  completed cycle %d in %s"%(cycle, \
      HMStime(time.time()-self.start_times['repX cycle'])))
    MC_report = " "
    for move_type in ['ExternalMC','SmartDarting','Sampler']:
      total_acc = np.sum(acc[move_type])
      total_att = np.sum(att[move_type])
      if total_att>0:
        MC_report += " %s %d/%d=%.2f (%.1f s);"%(move_type, \
          total_acc, total_att, float(total_acc)/total_att, \
          self.timings[move_type])
    MC_report += " repX t %.1f s"%self.timings['repX']
    self.tee(MC_report)



  sLJr grid loaded from LJr.nc in 0.40 s
  sELE grid loaded from pbsa.nc in 0.03 s
  LJr grid loaded from LJr.nc in 0.35 s
  LJa grid loaded from LJa.nc in 0.16 s
  ELE grid loaded from pbsa.nc in 0.02 s
  completed cycle 4 in 19.70 s
  ExternalMC 3224/7750=0.42 (4.0 s); Sampler 2092/3400=0.62 (41.0 s); repX t 4.3 s


In [10]:
acc_rates = np.array(acc['Sampler'],dtype=np.float)/att['Sampler']

In [11]:
if 'HamiltonianMonteCarlo' in self.sampler[process].__module__:
  acc_rates = np.array(acc['Sampler'],dtype=np.float)/att['Sampler']
  for k in range(K):
    acc_rate = acc_rates[k]
    if acc_rate>0.8:
      lambdas[k]['delta_t'] += 0.125*MMTK.Units.fs
      lambdas[k]['steps_per_trial'] = min(lambdas[k]['steps_per_trial']*2,\
        self.params[process]['steps_per_sweep'])
    elif acc_rate<0.4:
      if lambdas[k]['delta_t']<2.0*MMTK.Units.fs:
        lambdas[k]['steps_per_trial'] = max(int(lambdas[k]['steps_per_trial']/2.),1)
      lambdas[k]['delta_t'] -= 0.25*MMTK.Units.fs
      if acc_rate<0.1:
        lambdas[k]['delta_t'] -= 0.25*MMTK.Units.fs
    if lambdas[k]['delta_t']<0.1*MMTK.Units.fs:
      lambdas[k]['delta_t'] = 0.1*MMTK.Units.fs


62 0.003 50
67 0.003 50


In [14]:
self.params[process]['steps_per_sweep']

50

In [17]:
lambdas[k]['steps_per_trial'] = min(lambdas[k]['steps_per_trial']*2,\
  self.params[process]['steps_per_sweep'])


In [18]:
lambdas[k]['steps_per_trial']

50